In [ ]:
# 유튜브 라이브러리 설치
# !pip install google-api-python-client

In [ ]:
# 라이브러리 로드
import pandas as pd
from googleapiclient.discovery import build # google api 연결
from tqdm import tqdm # 진행상황 파악

import warnings # 경고창 무시
warnings.filterwarnings('ignore')

In [ ]:
def collect_comments (video_list, result_list, api_key, keyword) :
    
    #tqdm 적용
    video_list = tqdm(video_list) # 영상 항목 별로 진행상황 확인

    # api 연결
    api_obj = build ('youtube', 'v3', developerKey = api_key)
    
    # 각 video의 댓글이 잘 합쳐졌는지 확인하기 위해서 shape 생성
    shape_count = list()
    
    for video in video_list: # 리스트에 담긴 첫 번째 영상부터 마지막 영상까지 반복
        video_id = video[17:] # 영상 공유하기 기준 url의 video id는 18번째 인덱스부터 끝까지임(모든 영상 동일)
        video_list.set_description ("Collecting Comments from youtube....") # tqdm.set_description : 진행상황 확인 중 설명 메세지 달기
        response = api_obj.commentThreads().list(part='snippet,replies',
                                         videoId = video_id, 
                                        maxResults = 100). execute()
        
        # 각 비디오의 댓글 
        comments = list() # 댓글을 담을 빈 리스트 생성
        while response:
            for item in response['items']:
                # json 파일에서 불러올 항목 찾기
                comment = item['snippet']['topLevelComment']['snippet']
                # 댓글 내용, 작성자, 게시일, 좋아요 수 수집
                comments.append([comment['textDisplay'], comment['authorDisplayName'], comment['publishedAt'],comment['likeCount']])
                
                # 대댓글 수집 (대댓글 1~3개인 것들은 중간에 댓글이 삭제되거나 하면 값을 가져오지 못하는 오류가 발생해 4개부터 가져옴)
                # 댓글 수집과 같은 원리
                if item['snippet']['totalReplyCount'] > 3:
                    for reply_item in item['replies']['comments']:
                        reply = reply_item['snippet']
                        comments.append([reply])
                        comments.append([reply['textDisplay'], reply['authorDisplayName'], reply['publishedAt'], reply['likeCount']])

            # page 넘겨가면서 댓글 조회 
            if 'nextPageToken' in response:
                response = api_obj.commentThreads().list(part='snippet,replies', videoId=video_id, pageToken=response['nextPageToken'], maxResults=100).execute()
            else:
                break

        # 데이터 프레임으로 만들어주기 
        df = pd.DataFrame(comments, columns = ["comment", "author", "datetime", "like_count"])
        
        # na 값 제거
        df = df.dropna(axis=0)
        
        # 각 비디오별 자료개수 파악
        print(f"{video_id} :  {df.shape}")
        
        # 합계할 리스트에 삽입
        shape_count.append(df.shape[0])

        # 최종 df로 만들기 전에 리스트에 삽입
        result_list.append(df)
        
    # 최종 df로 합쳐주기 
    result_list = pd.concat(result_list,ignore_index=True)
    
    # 파일 생성
    result_list.to_csv(f"{keyword}.csv", index=False)

    if sum(shape_count) == result_list.shape[0]:
        print("합계가 일치합니다.")
        return result_list
    else:
        print(f"sum : {sum(shape_count)} / {result_list.shape[0]} 합계가 일치하지 않습니다.")
        return result_list

In [ ]:
url_list = ["https://youtu.be/mktb4KDO4Jo", "https://youtu.be/v9n3z9UEPXs"] # 영상 공유하기 기준 url
api_key = 'api key'
comments_list = [] # 댓글을 저장할 리스트 생성
keyword = "이강인"

collect_comments(url_list, comments_list, api_key, keyword)

mktb4KDO4Jo :  (1779, 4)


v9n3z9UEPXs :  (2913, 4)
합계가 일치합니다.


,comment,author,datetime,like_count
0,이강인처럼 받쳐주는애들이 있어야 손흥민같은 골잡이가 골을넣지...,ISFJ,2022-10-01T01:14:32Z,0.0
1,이런 선수를 벤투 새끼는 왜 기용을 거부하는건지?,짧고 굵게,2022-10-01T01:07:00Z,1.0
2,"누가 이강인 수비 못한다고 했냐? 아가리 뺨따구 찢어 불라 콱 씨;,.",DoSiGaS,2022-10-01T00:28:38Z,0.0
3,이런 애를 불러다가 안쓸거면 부르지라도 말지 벤투야.........경기력 폼 최상위...,Carlos Alberto Vela Garrido,2022-09-30T23:10:31Z,0.0
4,하지만 벤투에겐 벤치워머 그 이상도 이하도 아니지..... 하아.... 벤투 ...,Evan Kim,2022-09-30T22:45:10Z,0.0
...,...,...,...,...
4687,이제동 제일ㅈ같은건 var있어도 안쓴다는거,프로댓글러,2019-09-25T23:23:14Z,0.0
4688,"@이제동 거의없는데???<br>축구스타일,판정이 약간 다른거지",ᅡᅡ,2019-09-25T23:21:20Z,0.0
4689,유럽축구 실력은 인정합니다만 마인드는 썩었습니다 편파판정난무하여도 이기기만 하면 너...,이제동,2019-09-25T23:15:46Z,0.0
4690,장대장군 ㅇㅈ,프로댓글러,2019-09-25T23:13:33Z,0.0


In [ ]:
df = pd.read_csv(f"{keyword}.csv")
df.shape

(4710, 4)